# Libraries

In [1]:
%matplotlib inline
import cv2
import os

import numpy as np
import mediapipe as mp

from mp_support import mediapipe_detection, draw_landmarks, draw_styled_landmarks, extract_keypoints, extract_coordinates, prob_viz

# Import mediapipe model for finger and pose detection

In [2]:
mp_holistic = mp.solutions.holistic # holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

# Folder creation

Create folders for
1. Label folders for new actions
2. Sub-label folders

In [8]:
# base directory
directory_path = './augmented_data'
# current directory
c_dir = os.getcwd()

actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))])) # sorted to follow folder arrangement
# actions = np.array(['afternoon', 'house', 'again'])

In [9]:
actions

array(['afternoon', 'again', 'base', 'door', 'hello', 'house', 'how',
       'open', 'see', 'sorry', 'why'], dtype='<U9')

# Keypoint extraction
Here we extract the keypoints of the data (videos) by looping through each set of videos in each action folder, then writing the corresponding keypoints. 

This will be used later for model training.

In [10]:
video_count = 0 # required for processing later
for action in actions:
    counter = 0 # to count video/extracted file
    if not os.path.isdir(os.path.join(c_dir, 'labels_new', action)):
        os.mkdir(os.path.join(c_dir, 'labels_new', action))
    for video in os.listdir(os.path.join(directory_path, action)): # going through each converted video file in the action
        counter += 1
        video_count += 1
        filepath = os.path.join(os.path.join(c_dir, 'augmented_data', action, video))

        # NOTE: for each video, save keypoints in new subfolder
        # create subfolder if it does not exist
        subfolder = str(action) + '_' + str(counter)
        if not os.path.isdir(os.path.join(c_dir, 'labels_new', action, subfolder)):
            os.mkdir(os.path.join(c_dir, 'labels_new', action, subfolder))
        # set new savepath
        savepath = os.path.join(c_dir, 'labels_new', action, subfolder)
        print(f'Currently reading video {video_count}: {filepath}')
        extract_coordinates(filepath, savepath)


Currently reading video 1: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_001.mp4


c:\Users\wei-z\anaconda3\envs\slproj\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Currently reading video 2: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_002.mp4
Currently reading video 3: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_003.mp4
Currently reading video 4: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_004.mp4
Currently reading video 5: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_005.mp4
Currently reading video 6: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_006.mp4
Currently reading video 7: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_007.mp4
Currently reading video 8: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_008.mp4
Currently reading video 9: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_009.mp4
Currently reading video 10: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_010.mp4
Currently reading video 11: d:\GitHub\SSLreco

In [17]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))]))

In [13]:
import keras
import cv2

In [18]:
# Load pre-trained Keras model
lstm_model = keras.models.load_model('lstm_model.h5')

sequence = []
sentence = []
predictions = []
threshold = 0.3

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 267:
            res = lstm_model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti